Architecture of a FHIR app:
![image](https://cdn-images-1.medium.com/max/1200/1*qRd-H-cPHAGTB993sNLy3Q.png)
(from https://blog.heliossoftware.com/fhir-architectural-patterns-ae828b13d40c)

# FHIR-Drills (Simple Patient)

This notebook is a rough, Jupyter translation of the FHIR-Drills tutorial based on Postman avaialble [here](https://fhir-drills.github.io/simple-patient.html). Rather than using the Python Smart on FHIR library, I'm going to use the Python requests library so that we are working more directly with the HTTP protocol and are closer to the steps described in the original tutorial

## Find a patient with relevant variables

In [ ]:
import requests
from post_data import create_resource
import urllib

In [ ]:
FHIRJSONMimeType = 'application/fhir+json'
header_defaults = {
            'Accept': FHIRJSONMimeType,
            'Accept-Charset': 'UTF-8',
        }
URL = 'https://stu3.test.pyrohealth.net/fhir'

### Create Patient and get `id`

In [ ]:
patientId = create_resource()

In [ ]:
patientId

### Set the headers to work with FHIR data

To request the FHIR resource, we build up a URL based on

- The base URL: `URL`
- The resource type we want: `Patient`
- The ID of the resoruce we want: `patientID` (returned from `create_resource()`)



In [ ]:
rurl = URL+ '/Patient/'+patientId
rurl

## Load the data and get basic demographics

In [ ]:
r = requests.get(rurl,
                 headers = header_defaults)

## Get the returned content

We are going to extract the returned data and use the resposne objects `json` method to convert the returned object to a Python dictionary.

In [ ]:
patient = r.json()
print(type(patient))

In [ ]:
patient

Dictionary are __key__/__value__ pairs.

We can list the keys with the `keys()` method.

In [ ]:
patient.keys()

We access the values in the dictionary using square brackets and the keys.

For example,

In [ ]:
patient["birthDate"]

We can change the value by assigning a value to the key

In [ ]:
patient["birthDate"] = "1970-01-01"

## Change Patient name

The tutorial suggests we change the patient name to your own and then push the results up to the server.

In [ ]:
patient["name"] = [{'family': "Chapman", 'given':['Brian', 'Earl'], 
              "prefix":"Dr.", "text":"Brian Chapman", "use":"official"}]

## PUT the modified data on the server

we will use the `PUT` method to push our modified patient to the server.

In [ ]:
r2 = requests.put(URL+'/Patient/'+patientId,
                 headers = header_defaults,
                 json=patient)
r2.ok

## Verify Results

In [ ]:
requests.get(URL+'/Patient/'+patientId,
                 headers = header_defaults).json()['name']

## Replicate using the fhirclient

In [ ]:
from fhirclient import client
print('FHIR client library has been loaded')

In [ ]:
settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://stu3.test.pyrohealth.net/fhir'
}

In [ ]:
# Create an instance of the FHIR client that points to this FHIR server. We use 
# this to make the calls.
db = client.FHIRClient(settings=settings)

In [ ]:
# Perform a GET (read) on the patient with the ID you identified earlier
from fhirclient.models.patient import Patient
patient = Patient.read(patientId, db.server)

In [ ]:
p = patient.as_json()

In [ ]:
p

## *Optional: get basic demographics*

In [ ]:
# Patient name. 
# Show the name object so we can see what's in it
print(patient.name)

The patient's name is a list of `HumanName`s. We can find out about the structure of this data type on the FHIR website: https://www.hl7.org/fhir/datatypes.html#HumanName

In [ ]:
# Show the contents of each object in the list
for name in patient.name:
    print(name.as_json())

In [ ]:
# We see that a Patient can have more than one name (i.e. Patient.name is a list)
# and each name can have more than one given name (i.e. given is also a list).
# Let's use the first name in the list and the first given name:
first_name = patient.name[0].given[0]
last_name = patient.name[0].family
print(first_name, last_name)

In [ ]:
# Patient's gender
gender = patient.gender
gender

In [ ]:
# Define variable dob as the patient's date of birth
dob = patient.birthDate.date
print(dob)

In [ ]:
# We'd like to know the patient's age. The EHR contains the date of birth only, 
# so we have to calculate the age using the dob and today's date.
# For this we need the datetime library
import datetime

# Define and print today variable.
today = datetime.date.today()
print(today)

In [ ]:
# Calculate the patient's age using the relativedelta method of the dateutil 
# module
from dateutil.relativedelta import relativedelta

delta = relativedelta(today, dob)
age = delta.years
age

In [ ]:
# Print patient name, gender, DOB, age, and today's date
print("Patient's Name(s) =", first_name, last_name)
print("Gender =", gender)
print("DOB =", dob)
print("Today's Date =", today)
print("Patient's age =", age)